<h1>Procesing tmdb_All_movies.csv (Movies)</h1>

In [2]:
import csv, json, gensim, datetime, time
import nltk
import re
import gensim
import numpy as np
import pandas as pd
from gensim import corpora

rowslist = []
genreslist = []
productioncompanieslist = []
overviewlist = []

stop_list = nltk.corpus.stopwords.words('english')
stemmer = nltk.stem.porter.PorterStemmer()

def readJSON(row, rownumber, list, keyword):
    jsonstring = row[rownumber]

    jsonobj = json.loads(jsonstring)

    for jsonelement in jsonobj:
        keywordelement = jsonelement[keyword]       
        list.append(keywordelement.lower()) 
    
    return list
    
def processText(row, rownumber1, rownumber2, list):
    text = row[rownumber1].split(" ") + row[rownumber2].split(" ") #contains overview and tagline
    
    text1 = [w.lower() for w in text]
    text2 = [w for w in text1 if re.search('^[a-z]+$', w)]
    text3 = [w for w in text2 if w not in stop_list]
    text4 = [stemmer.stem(w) for w in text3]  
    
    list = text4
    
    return list
    
#read file
with open('tmdb_All_movies.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:        
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            elif line_count != 1000:
                eachnewrow = []
                genres = []
                productioncompanies = []
                productioncountries = []
                overview = []

                eachnewrow.append(row[0]) #budget

                if row[1] != "":
                    #handle json genres
                    genres = readJSON(row, 1, genres, 'name')

                eachnewrow.append(genres) #genres
                genreslist.append(genres)

                #row[2] homepage removed
                eachnewrow.append(row[3]) #id

                #row[4] original_language removed
                #row[5] original_title removed  
                                            
                #handle overview
                overview = processText(row, 6, 15,overview)

                eachnewrow.append(overview)
                overviewlist.append(overview)

                eachnewrow.append(row[7]) #popularity                  
                
                if row[8] != "":
                    #handle json productioncompanies
                    productioncompanies = readJSON(row, 8, productioncompanies, 'name')

                eachnewrow.append(productioncompanies) #productioncompanies
                productioncompanieslist.append(productioncompanies)                  

                #row[9] productioncountries

                weekday = ""
                day = ""
                month = ""
                year = ""                 

                #handle releasedate 
                if row[10] != "":
                    d = datetime.datetime.strptime(row[10] ,'%Y-%m-%d')

                    weekdayword = d.strftime('%a')
                    weekday = time.strptime(weekdayword, '%a').tm_wday + 1

                    day = d.strftime('%d')

                    monthword = d.strftime('%b')
                    month = time.strptime(monthword,'%b').tm_mon

                    year = d.strftime('%Y')

                eachnewrow.append(weekday) #weekday
                eachnewrow.append(day) #day 
                eachnewrow.append(month) #month
                eachnewrow.append(year) #year

                eachnewrow.append(row[11]) #revenue
                eachnewrow.append(row[12]) #runtime
                #row[13] spoken_languages removed  
                eachnewrow.append(row[14]) #status                      
                #row[15] tagline added to overview
                eachnewrow.append(row[16]) #title
                eachnewrow.append(row[17]) #vote_average
                eachnewrow.append(row[18]) #vote_count                  

                WR = (float(row[18]) / (float(row[18])+25000)) * float(row[17]) + (25000 / (float(row[18]) + 25000)) * 7.0            

                eachnewrow.append(WR) #weighted_rating

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception' + str(e))    
            line_count += 1          
    print(f'Processed {line_count} lines.')   
                  
genresdictionary = corpora.Dictionary(genreslist)
token_to_id_genres = genresdictionary.token2id                

overviewdictionary = corpora.Dictionary(overviewlist)
token_to_id_overview = overviewdictionary.token2id                            
                      
productioncompaniesdictionary = corpora.Dictionary(productioncompanieslist)
token_to_id_productioncompanies = productioncompaniesdictionary.token2id                  
                      
def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros

tablecolumns = ['budget'] + list(genresdictionary.values()) + ['id'] + list(overviewdictionary.values()) + ['popularity'] + list(productioncompaniesdictionary.values()) + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'status', 'title', 'vote_average', 'vote_count', 'weighted_rating']

tablerows = []
                      
count = 1
for eachrow in rowslist:
    listofzerosgenres = [0] * len(token_to_id_genres)
    listofzerosoverview = [0] * len(token_to_id_overview)                          
    listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)                    

    vec_genres = genresdictionary.doc2bow(eachrow[1])
    vec_overview = overviewdictionary.doc2bow(eachrow[3])                         
    vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[5])                     

    listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
    listofzerosoverview = assigningValues(vec_overview, listofzerosoverview)                      
    listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)                      
    
    tablerows.append(eachrow[0:1] + listofzerosgenres + eachrow[2:3] + listofzerosoverview + eachrow[4:5] 
                     + listofzerosproductioncompanies + eachrow[6:17])

    print(count)
    count+=1     
                      
df = pd.DataFrame(np.array(tablerows), columns=tablecolumns)                      
print(df.head(10))      
#write to file
""""
with open('processed_tmdb_movies.csv', mode='w', newline='', encoding='utf-8') as movies_file: #change accordingly
    writer2 = csv.writer(movies_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer2.writerow(['budget'] + list(genresdictionary.values()) + ['id'] + list(overviewdictionary.values()) + ['popularity'] 
        + list(productioncompaniesdictionary.values()) 
        + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'status', 'title', 'vote_average', 'vote_count', 'weighted_rating'])              
    
    for eachrow in rowslist:
        listofzerosgenres = [0] * len(token_to_id_genres)
        listofzerosoverview = [0] * len(token_to_id_overview)                          
        listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)                    
                  
        vec_genres = genresdictionary.doc2bow(eachrow[1])
        vec_overview = overviewdictionary.doc2bow(eachrow[3])                         
        vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[5])                     

        listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
        listofzerosoverview = assigningValues(vec_overview, listofzerosoverview)                      
        listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)

        writer2.writerow(eachrow[0:1] + listofzerosgenres + eachrow[2:3] + listofzerosoverview + eachrow[4:5] 
        + listofzerosproductioncompanies + eachrow[6:17])
"""
print('\nOutput Success!')                 

Column names are budget, genres, homepage, id, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count
Processed 1000 lines.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210

<h1>Processing tmdb_All_credits.csv (Credits)</h1>

In [3]:
import csv, json, gensim
import pandas as pd
from gensim import corpora

rowslist = []
castslist = []
crewslist = []

#read file
with open('tmdb_All_credits.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            elif line_count != 1000:
                eachnewrow = []
                casts = []
                crews = []

                eachnewrow.append(row[0]) #id

                #handle json casts
                if row[1] != "":                      
                    jsonstring = row[1]

                    jsonobj = json.loads(jsonstring)

                    for jsonelement in jsonobj:
                        cast = jsonelement['name']       
                        casts.append(cast.lower())

                eachnewrow.append(casts) #casts
                castslist.append(casts)

                #handle json crews 
                if row[2] != "":                       
                    jsonstring2 = row[2]

                    jsonobj2 = json.loads(jsonstring2)

                    for jsonelement in jsonobj2:
                        job = jsonelement['job']
                        if job == "Director":
                            crew = jsonelement['name']       
                            crews.append(crew.lower())

                eachnewrow.append(crews) #crews
                crewslist.append(crews)         

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception' + str(e))    
            line_count += 1   

    print(f'Processed {line_count} lines.')
                  
castsdictionary = corpora.Dictionary(castslist)
token_to_id_casts = castsdictionary.token2id
                  
crewsdictionary = corpora.Dictionary(crewslist)
token_to_id_crews = crewsdictionary.token2id                  

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros                  

tablecolumns2 = ['id'] + list(castsdictionary.values()) + list(crewsdictionary.values())
                      
tablerows2 = []        

count = 1
for eachrow in rowslist:
    listofzeroscasts = [0] * len(token_to_id_casts)
    listofzeroscrews = [0] * len(token_to_id_crews)                         

    vec_casts = castsdictionary.doc2bow(eachrow[1])
    vec_crews = crewsdictionary.doc2bow(eachrow[2])   

    listofzeroscasts = assigningValues(vec_casts, listofzeroscasts)
    listofzeroscrews = assigningValues(vec_crews, listofzeroscrews)

    tablerows2.append(eachrow[0:1] + listofzeroscasts + listofzeroscrews) 
                      
    print(count)
    count+=1                      
                      
df2 = pd.DataFrame(np.array(tablerows2), columns=tablecolumns2)                      
print(df2.head(10))                        
"""                     
#write to file
with open('processed_tmdb_credits.csv', mode='w', newline='', encoding='utf-8') as credits_file: #change accordingly
    writer = csv.writer(credits_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['id'] + list(castsdictionary.values()) + list(crewsdictionary.values()))              
    
    for eachrow in rowslist:
        listofzeroscasts = [0] * len(token_to_id_casts)
        listofzeroscrews = [0] * len(token_to_id_crews)                         
                  
        vec_casts = castsdictionary.doc2bow(eachrow[1])
        vec_crews = crewsdictionary.doc2bow(eachrow[2])   

        listofzeroscasts = assigningValues(vec_casts, listofzeroscasts)
        listofzeroscrews = assigningValues(vec_crews, listofzeroscrews)

        writer.writerow(eachrow[0:1] + listofzeroscasts + listofzeroscrews)
"""
print('\nOutput Success!')                  

Column names are id, cast, crew
Processed 1000 lines.
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
26

<h1>Merging Two Data Frames</h1>

In [5]:
import pandas as pd
import pickle

df3 = df.join(df2, lsuffix='id', rsuffix='id')
print(df3.head(10)) 

df3.to_pickle("processed_data")

#df3 = pd.read_pickle("processed_data")

"""
movies = pd.read_csv("processed_tmdb_movies.csv") #change accordingly
credits = pd.read_csv("processed_tmdb_credits.csv") #change accordingly
merged = pd.merge(movies, credits, left_on=['id'],
                   right_on=['id'], 
                   how='left')
merged.to_csv("processed_tmdb_data.csv", index=False)
"""
print('\nOutput Success!')  

     budget music drama horror documentary thriller comedy crime western  \
0         0     0     0      0           0        0      0     0       0   
1  17000000     0     0      0           0        0      0     0       0   
2         0     0     0      0           0        0      0     0       0   
3         0     1     0      0           0        0      0     0       0   
4         0     0     1      0           0        0      0     0       0   
5   3500000     0     0      1           0        0      0     0       0   
6         0     0     0      0           1        0      0     0       0   
7         0     0     0      0           0        0      0     0       0   
8         0     0     0      0           0        0      0     0       0   
9         0     0     0      0           0        0      0     0       0   

  romance     ...      chris dowling ian bonhôte afdhere jama chloe jackson  \
0       0     ...                  0           0            0             0   
1    

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('tmdb_All_movies.csv')

df['revenue'] = df['revenue'].astype('uint64')
#sortdf = df.sort_values(df['revenue'], ascending=[1,0])
revenuearray = df['revenue'].tolist()

sortedrevenue = revenuearray.sort()

length = len(revenuearray)
print(length)
arrayrange = []
binsize = int(length/5)
minvalue=0

for i in range(0,5): #9 bin
    minvalue = i * binsize
    arrayrange.append([revenuearray[minvalue],revenuearray[(i+1)*binsize-1]])
    print(minvalue)

print(arrayrange)    

print(revenuearray[0:binsize])
print("-----")
print(revenuearray[binsize:(binsize*2)])

23579
0
4715
9430
14145
18860
[[1, 46296], [46296, 1000000], [1000000, 10623500], [10623500, 47267829], [47267829, 2187463944]]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15,